In [122]:
# Initialize notebook
import pandas as pd
import numpy as np

METADATA = {'Contributor': 'T. Dunn'}

SAVEFIGS = False

DAMAGE_COLUMNS = [
    'damage immunities',
    'damage resistances',
    'damage vulnerabilities',
    'damage modifiers.acid',
    'damage modifiers.bludgeoning',
    'damage modifiers.cold',
    'damage modifiers.fire',
    'damage modifiers.force',
    'damage modifiers.lightning',
    'damage modifiers.necrotic',
    'damage modifiers.piercing',
    'damage modifiers.poison',
    'damage modifiers.psychic',
    'damage modifiers.radiant',
    'damage modifiers.slashing',
    'damage modifiers.thunder',
]

df0 = pd.read_csv('monster-data.csv')
df0 = df0.astype({
    'size': 'category',
    'source.book': 'category', 
    'source.page': 'Int32',
    'type': 'category',
})

for c in DAMAGE_COLUMNS:
    df0[c] = df0[c].fillna('')
    df0[c] = df0[c].astype('category')

print('Includes {} monsters in database'.format(len(df0.index.to_list())))
df0.dtypes

Includes 1920 monsters in database


source.book                     category
source.page                        Int32
size                            category
type                            category
cr                               float64
legacy                              bool
damage immunities               category
damage resistances              category
damage vulnerabilities          category
damage modifiers.acid           category
damage modifiers.bludgeoning    category
damage modifiers.cold           category
damage modifiers.fire           category
damage modifiers.force          category
damage modifiers.lightning      category
damage modifiers.necrotic       category
damage modifiers.piercing       category
damage modifiers.poison         category
damage modifiers.psychic        category
damage modifiers.radiant        category
damage modifiers.slashing       category
damage modifiers.thunder        category
dtype: object

In [123]:
# Setup template and html config for plotly figures.
import plotly.graph_objects as go

TFB_TEMPLATE = dict(
    layout=go.Layout(
        template='plotly_white',
        autosize=True, # must be True to auto-scale when resizing
        margin=dict(l=50, r=25, b=40, t=20, pad=4),
        font=dict(
            family='sans-serif',
            size=14
        ),
        hovermode='closest',
        xaxis=dict(
            #autorange=False,
            automargin=True,
            #fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        yaxis=dict(
            #autorange=False,
            automargin=True,
            #fixedrange=True,
            showline=True,
            linecolor='#444',
            linewidth=2,
            mirror=True,
            #tickmode='linear',
            ticks='outside',
            minor=dict(
                #showgrid=True,
                #tickmode='linear',
                ticks='outside',
            ),
            zeroline=False,
        ),
        hoverlabel=dict(align='left'),
        #legend=dict(font_size=10),
    )
)

TFB_CONFIG = {
    'responsive': True, # must be True to auto-scale when resizing
    'autosizable': True, # doesn't impact auto rescaling
    'showAxisDragHandles': False,
    'displaylogo': False,
    'displayModeBar': 'hover',
    'modeBarButtonsToRemove': [
        'select2d',
        'lasso2d',
        'zoom2d',
        'zoomIn2d',
        'zoomOut2d',
        'pan2d',
        'autoScale2d',
        'hoverClosestCartesian',
        'hoverCompareCartesian',
        'toggleSpikelines',
        'resetScale2d',
    ],
    'toImageButtonOptions': {
        'format': 'png', # one of png, svg, jpeg, webp
        'filename': 'tfb-plot',
        'height': 450,
        'width': 600,
        'scale': 2
    },
}


In [124]:
# define custom functions
DAMAGE_COLUMNS_NONPHYSICAL = [
    'damage modifiers.acid',
    'damage modifiers.cold',
    'damage modifiers.fire',
    'damage modifiers.force',
    'damage modifiers.lightning',
    'damage modifiers.necrotic',
    'damage modifiers.poison',
    'damage modifiers.psychic',
    'damage modifiers.radiant',
    'damage modifiers.thunder',
]

DAMAGE_COLUMNS_PHYSICAL = [
    'damage modifiers.bludgeoning',
    'damage modifiers.piercing',
    'damage modifiers.slashing',
]

DAMAGE_COLUMNS = DAMAGE_COLUMNS_NONPHYSICAL + DAMAGE_COLUMNS_PHYSICAL

NONMAGIC_DECODER = {
    '': 1.0,
    'I': 0.0,
    'I-F': 0.0,
    'I-N': 0.0,
    'I-NA': 0.0,
    'I-NS': 0.0,
    'R': 0.5,
    'R-D': 0.5,
    'R-M': 0.5,
    'R-N': 0.5,
    'R-NA': 0.5,
    'R-ND': 0.5,
    'R-NS': 0.5,
    'V': 2.0,
}

MAGIC_DECODER = {
    '': 1.0,
    'I': 0.0,
    'I-F': 0.0,
    'I-N': 1.0,
    'I-NA': 1.0,
    'I-NS': 1.0,
    'R': 0.5,
    'R-D': 0.5,
    'R-M': 0.5,
    'R-N': 1.0,
    'R-NA': 1.0,
    'R-ND': 1.0,
    'R-NS': 1.0,
    'V': 2.0,
}

def damage_modifier_count( row, modifier ):
    """Returns the number of times a monster has the specified damage modifier

        modifier: Str representing the type of damage modifier.
    """
    match modifier.lower():
        case 'resistant':
            m = 'R'
        case 'immune':
            m = 'I'
        case 'vulnerable':
            m = 'V'
        case _ :
            return False

    cnt = 0
    for dc in DAMAGE_COLUMNS:
        if len(row[dc]) == 0:
            continue
        elif row[dc][0:1] == m:
            cnt += 1
    return cnt

def has_damage_modifier( row, modifier ):
    """Returns True if monster has the specified damage modifier

        modifier: Str representing the type of damage modifier.
    """
    return damage_modifier_count(row, modifier) > 0

def average_damage_reduction( row, decoder ):
    dr = 0
    for dt in DAMAGE_COLUMNS:
        dr += decoder[row[dt]]
    dr /= len(DAMAGE_COLUMNS)
    return (1 - dr)

In [116]:
# plots average damage modifier for magical damage types vs CR
import plotly.graph_objects as go
from bs4 import BeautifulSoup
from scipy.interpolate import UnivariateSpline

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        barmode='stack',
        yaxis=dict(
            title_text='percent of monsters',
            range=[0, 35],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.01, y=0.99,
            orientation='v',
            tracegroupgap=0,
            traceorder='normal',
        )
    )
)

# plot data
df1 = df0[df0['cr'].between(1,30) & df0['legacy'].eq(False)]
x = np.array([column.split('.')[1] for column in DAMAGE_COLUMNS_NONPHYSICAL])
t = np.array(len(x)*[df1.shape[0]])
for group in ['resistant','immune','vulnerable']:
    y = []
    for column in DAMAGE_COLUMNS_NONPHYSICAL:
        temp = [r[0:1] if r else 'N' for r in df1[column]]
        y.append(np.sum([1 if r == group[0:1].upper() else 0 for r in temp]))
    y = np.array(y)
    text = [f'{ty} out of {tt}' for ty, tt in zip(y, t)]
    fig.add_trace(go.Bar(
        x=x,
        y=100*y/t,
        name=group,
        text=text,
        textposition='none',
        hovertemplate=
            '<b>' + group + ' to %{x}</b><br>'+
            '%{y:.1f}% (%{text})'+
            '<extra></extra>'
    ))

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-nonphysical-modifier-types-by-damage-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_layout(
        font=dict(size=10),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.01, y=1.04,
            orientation='h',
        )
    )
    file_name = f'./fig-nonphysical-modifier-types-by-damage-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio: 600/500;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

In [22]:
# plots average damage modifier for magical damage types vs CR
import plotly.graph_objects as go
from bs4 import BeautifulSoup
from scipy.interpolate import UnivariateSpline

NONMAGIC_DECODER = {
    '': 1.0,
    'I': 0.0,
    'I-F': 0.0,
    'I-N': 0.0,
    'I-NA': 0.0,
    'I-NS': 0.0,
    'R': 0.5,
    'R-D': 0.5,
    'R-M': 0.5,
    'R-N': 0.5,
    'R-NA': 0.5,
    'R-ND': 0.5,
    'R-NS': 0.5,
    'V': 2.0,
}

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        barmode='stack',
        hovermode='x unified',
        yaxis=dict(
            title_text='percent of monsters',
            range=[0, 35],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='right', yanchor='top',
            x=0.99, y=0.99,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# determine the sort order
y = np.zeros(len(DAMAGE_COLUMNS_NONPHYSICAL))
for i in range(len(DAMAGE_COLUMNS_NONPHYSICAL)):
    column = DAMAGE_COLUMNS_NONPHYSICAL[i]
    y[i] = np.mean(df1.apply(lambda row: NONMAGIC_DECODER[row[column]], axis=1))
ind = np.argsort(y)
damage_columns = [DAMAGE_COLUMNS_NONPHYSICAL[i] for i in ind]

# plot data
df1 = df0[df0['cr'].between(1,30) & df0['legacy'].eq(False)]
x = np.array([column.split('.')[1] for column in damage_columns])
t = np.array(len(x)*[df1.shape[0]])
for group in ['resistant','immune','vulnerable']:
    y = []
    for column in damage_columns:
        temp = [r[0:1] if r else 'N' for r in df1[column]]
        y.append(np.sum([1 if r == group[0:1].upper() else 0 for r in temp]))
    y = np.array(y)
    text = [f'{ty} out of {tt}' for ty, tt in zip(y, t)]
    fig.add_trace(go.Bar(
        x=x,
        y=100*y/t,
        name=group,
        text=text,
        textposition='none',
        hovertemplate=
            '%{y:.1f}% ' + group +
            '<extra></extra>'
    ))

# show figure
fig.show(config=TFB_CONFIG)

In [120]:
# plots average damage modifier for magical damage types vs CR
import plotly.graph_objects as go
from bs4 import BeautifulSoup
from scipy.interpolate import UnivariateSpline

DAMAGE_TYPES = [
    dict(column='damage modifiers.acid',      color='limegreen'),
    dict(column='damage modifiers.cold',      color='dodgerblue'),
    dict(column='damage modifiers.fire',      color='red'),
    dict(column='damage modifiers.force',     color='teal'),
    dict(column='damage modifiers.lightning', color='gold'),
    dict(column='damage modifiers.necrotic',  color='darkviolet'),
    dict(column='damage modifiers.poison',    color='olivedrab'),
    dict(column='damage modifiers.psychic',   color='purple'),
    dict(column='damage modifiers.radiant',   color='orange'),
    dict(column='damage modifiers.thunder',   color='sienna'),
]

def plot_scatter_spline(fig, x, y, group, color, text, smooth_factor):
    fig.add_trace(go.Scatter(
        x=x,
        y=y,
        mode='markers', 
        name=group,
        marker=dict(size=4, color=color),
        showlegend=False,
        legendgroup=group,
        text=text,
        hovertemplate=
            'CR %{x}<br>'+
            '%{y:.0f}% ' + group + ' damage taken'+
            '<extra></extra>'
    ))

    # add fit line
    spl = UnivariateSpline(x, y)
    xs = np.linspace(1, 30, 1000)
    spl.set_smoothing_factor(smooth_factor)
    fig.add_trace(go.Scatter(
        x=xs,
        y=spl(xs),
        mode='lines', 
        line=dict(color=color, dash='solid'),
        showlegend=True,
        name=group,
        legendgroup=group,
        hoverinfo='skip',
    ))
    return fig

fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        xaxis=dict(
            range=[0.5,30.5],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='percent damage taken',
            range=[-2, 105],
            tick0=0, dtick=10,
            minor=dict(tick0=0, dtick=5),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# filter dataframe
df1 = df0[df0['cr'].between(1,30) & df0['legacy'].eq(False)]
dfG = df1.groupby(['cr']).mean().reset_index()
x = dfG['cr'].to_numpy()

# plot data for non-physical damage types
for dt in DAMAGE_TYPES:
    column = dt['column']
    color = dt['color']
    group = column.split('.')[1]
    df1['modifier'] = df1.apply(lambda row: NONMAGIC_DECODER[row[column]], axis=1)
    y = 100*(df1.groupby(['cr']).mean()['modifier'].to_numpy())
    fig = plot_scatter_spline(fig, x, y, group, color, '~', 1e6)
    print('{:} - {:.1%}'.format(group, 1 - np.mean(df1['modifier'])))

# plot data for physical damage types
df1['modifier'] = 0
for column in DAMAGE_COLUMNS_PHYSICAL:
    df1['modifier'] += (1/3)*df1.apply(lambda row: NONMAGIC_DECODER[row[column]], axis=1)
y = 100*(df1.groupby(['cr']).mean()['modifier'].to_numpy())
fig = plot_scatter_spline(fig, x, y, 'physical', 'gray', '~', 1e6)
print('{:} - {:.1%}'.format('physical', 1 - np.mean(df1['modifier'])))

# plot data for physical magical damage types
"""df1['modifier'] = 0
for column in DAMAGE_COLUMNS_PHYSICAL:
    df1['modifier'] += (1/3)*df1.apply(lambda row: MAGIC_DECODER[row[column]], axis=1)
y = 100*(df1.groupby(['cr']).mean()['modifier'].to_numpy())
fig = plot_scatter_spline(fig, x, y, 'physical (magical)', 'darkgray', '~')
print('{:} - {:.1%}'.format('physical (magical)', 1 - np.mean(df1['modifier'])))"""

# show figure
fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-average-modifier-vs-cr-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_traces(marker=dict(size=4))
    fig.update_layout(
        font=dict(size=10),
        legend=dict(font=dict(size=10), x=0.01, y=1.04, orientation='h'),
    )
    file_name = f'./fig-average-modifier-vs-cr-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'aspect-ratio: 600/550;'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

acid - 5.5%
cold - 11.3%
fire - 11.1%
force - 0.4%
lightning - 8.1%
necrotic - 7.4%
poison - 28.8%
psychic - 5.2%
radiant - 1.0%
thunder - 2.0%
physical - 12.3%


In [126]:
# Average damage reduction for each combination of damage and monster type

DAMAGE_TYPES = [
    dict(column='damage modifiers.acid',        color='limegreen'),
    dict(column='damage modifiers.cold',        color='dodgerblue'),
    dict(column='damage modifiers.fire',        color='red'),
    dict(column='damage modifiers.force',       color='teal'),
    dict(column='damage modifiers.lightning',   color='gold'),
    dict(column='damage modifiers.necrotic',    color='darkviolet'),
    dict(column='damage modifiers.poison',      color='olivedrab'),
    dict(column='damage modifiers.psychic',     color='purple'),
    dict(column='damage modifiers.radiant',     color='orange'),
    dict(column='damage modifiers.thunder',     color='sienna'),
    dict(column='damage modifiers.bludgeoning', color='darkgray'),
    dict(column='damage modifiers.piercing',    color='gray'),
    dict(column='damage modifiers.slashing',    color='dimgray'),
]

def damage_type_is_physical(dt):
    if dt.split('.')[1].lower() in ['bludgeoning','piercing','slashing']:
        return True
    else:
        return False

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=TFB_TEMPLATE,
        margin=dict(l=0, r=10, b=0, t=30, pad=4),
        font=dict(size=12),
        xaxis=dict(
            title_text='',
            autorange=True,
            showgrid=False,
        ),
        yaxis=dict(
            title_text='',
            autorange=True,
            showgrid=False,
        ),
    )
)

x = [t.lower() for t in df0['type'].cat.categories.to_list()]
y = [dt['column'].split('.')[1].lower() for dt in DAMAGE_TYPES]
z = []
hovertext = []
for dt in DAMAGE_TYPES:
    column = dt['column']
    df1 = df0[df0['cr'].between(1,30) & df0['legacy'].eq(False)][['type',column]]
    if damage_type_is_physical(column):
        df1['modifier'] = df1.apply(lambda row: 100*(1 - NONMAGIC_DECODER[row[column]]), axis=1)
    else:
        df1['modifier'] = df1.apply(lambda row: 100*(1 - NONMAGIC_DECODER[row[column]]), axis=1)
    dfG = df1.groupby(['type']).mean().reset_index()
    row = dfG['modifier'].tolist()
    z.append(row)
    type_totals = df0[df0['cr'].between(1,30) & df0['legacy'].eq(False)].groupby(['type']).count().reset_index()['source.book'].to_list()
    hovertext.append([str(tt) for tt in type_totals])

# generate heat map
fig.add_trace(go.Heatmap(
    z=z, x=x, y=y,
    zmin=0.0, zmax=80.0,
    hoverongaps=False,
    colorscale='blues',
    texttemplate='%{z:.0f}',
    text=hovertext,
    hovertemplate='<b>Monster:</b> %{x} (n=%{text})<br>'+
                  '<b>Damage:</b> %{y}<br>'+
                  '<b>Mitigation:</b> %{z:.1f}%'+
                  '<extra></extra>'
))

fig.update_traces(showscale=False)

fig.show(config=TFB_CONFIG)

# save large format figure
if SAVEFIGS:
    file_name = f'./fig-damage-reduction-heat-map-large.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    # add style to figure
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-large'
    fig_soup.div['style'] = 'width:1000px; min-width:50%; max-width:100%; max-height:500px'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))

# save small format figure
if SAVEFIGS:
    fig.update_layout(font_size=10)
    file_name = f'./fig-damage-reduction-heat-map-small.html'
    fig_html = fig.to_html(
        include_plotlyjs=False, 
        full_html=False, 
        config=TFB_CONFIG
    )
    # add style to figure
    fig_soup = BeautifulSoup(fig_html, 'html.parser')
    fig_soup.div['class'] = 'plotly-div-small'
    fig_soup.div['style'] = 'width:1000px; min-width:50%; max-width:100%; max-height:500px'
    with open(file_name, 'wb') as fout:
        fout.write(fig_soup.prettify('utf-8'))